# Real-time Valuation

## Introduction

This notebook demonstrates how to configure LUSID to perform a [*GetValuation*](https://www.lusid.com/docs/api/#operation/GetValuation) call to retrieve real-time prices from SIX.

### Instrument universe

This notebook requires an instrument universe to be populated in a file at the location `data/real-time-valuation/SIX-CROSSREFERENCE.csv`.  The `template` file in the same directory indicates the required columns.  The easiest way to create this file is using the CROSSREFERENCE dataset provided by SIX.  This can be accessed using Luminesce using a query like below in a LUSID environment which is entitled to access SIX data.

For each instrument you will need a identifier and the BC (exchange) code.

```
values
  ('1222171'),                -- A set of instrument identifiers
  ('2340545');

@dataQuery = select
  'CH' AS identifierScheme,  -- SIX-specific identifier type code
  column1 AS identifier,
  '4' as bc                  -- SIX-specific exchange code
from @data;

Select * from Six.Flex where IncludeDataSetId = TRUE
AND IncludeStatus = TRUE
AND PackageId = 'CrossReference'
AND Request = @dataQuery;

```

In [1]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import random
from datetime import datetime, timezone
from IPython.core.display import HTML
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

INFO:lusid.extensions.socket_keep_alive:Setting socket settings for Darwin (macOS)
INFO:lusid.extensions.socket_keep_alive:Setting socket settings for Darwin (macOS)
INFO:lusid.extensions.socket_keep_alive:Setting socket settings for Darwin (macOS)


LUSID Environment : https://lusid-pms-demo.lusid.com/docs


,apiVersion,buildVersion,excelVersion,links
0,v0,0.6.14750.0,0.5.3666,"{'relation': 'RequestLogs', 'href': 'https://l..."


In [2]:
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
portfolios_api = api_factory.build(lu.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.TransactionPortfoliosApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.QuotesApi)
instruments_api = api_factory.build(lu.InstrumentsApi)

In [ ]:
scope = "realtime-valuation-six"
portfolio_code = "portfolio"
recipe_code = "SixStreaming-Valoren"
default_currency = "GBP"

quotes_supplier = "Lusid"
quotes_field = "mid"
quotes_instrument_id = "ClientInternal"

## 1. Setup

The SIX realtime pricing feed relies on at least one of SIX-specific instrument identifiers being configured in your LUSID instance.  If the cell below shows an error then please contact support@lusid.com to request the additional instrument identifiers be added to your account.

In [4]:
required_id_types = set([
    'SixIsin_BC',
    'SixValoren_BC',
    'SixSedol_BC',
    'SixCusip_BC',
    'SixTicker_BC'
])

instrument_id_types = instruments_api.get_instrument_identifier_types()
domain_id_types = set([id.identifier_type for id in instrument_id_types.values if id.identifier_type.endswith("_BC")])

missing_id_types = required_id_types - domain_id_types

if len(missing_id_types) > 0:
    print(f"The following SIX Instrument identifier types need to be setup in {api_url}: {missing_id_types}")

## 2. Load Data

### 2.1 Instruments 

In [5]:
def add_BC(row, identifier_type):
    if not pd.isnull(row[identifier_type]):
        return str(row[identifier_type]) + '_' + str(row['BC'])

    return None

# Read a standard SIX CROSSREFERENCE file
# In real use case the user is expected to provide the SIX-CROSSREFERENCE.csv file based on client-specific licences
# When running in a pipeline the file won't exist to revert to the (empty) template
filename = "data/real-time-valuation/SIX-CROSSREFERENCE.csv"

if os.path.isfile(filename):
  instruments_df = pd.read_csv(filename)
else:
  instruments_df = pd.read_csv( "data/real-time-valuation/SIX-CROSSREFERENCE.template.csv")

# Add required columns
instruments_df["ISIN_BC"] = instruments_df.apply(lambda row: add_BC(row, 'ISIN'), axis=1)
instruments_df['tradingSymbol_BC'] = instruments_df.apply(lambda row: add_BC(row, 'tradingSymbol'), axis=1)
instruments_df['swissValorNumber_BC'] = instruments_df.apply(lambda row: add_BC(row, 'swissValorNumber'), axis=1)
instruments_df['multipleSEDOL_BC'] = instruments_df.apply(lambda row: add_BC(row, 'multipleSEDOL'), axis=1)
instruments_df['USCUSIP_BC'] = instruments_df.apply(lambda row: add_BC(row, 'USCUSIP'), axis=1)

#instruments_df

Create a mapping schema for the instruments using the provided FIGIs as the instrument identifiers. The instruments file is loaded into LUSID. 

In [6]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "swissValorNumber",
        "Figi" : "FIGIGlobalShareClassId",
        "Isin" : "ISIN",
        "SixIsin_BC" : "ISIN_BC",
        "Ticker" : "tradingSymbol",
        "SixTicker_BC" : "tradingSymbol_BC",
        "SixValoren_BC" : "swissValorNumber_BC",
        "Sedol" : "multipleSEDOL",
        "SixSedol_BC" : "multipleSEDOL_BC",
        "Cusip" : "USCUSIP",
        "SixCusip_BC" : "USCUSIP_BC",
    },
    "required": {
        "name": "FISNSIX"
    },
}

# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    instrument_scope=scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = cocoon.format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

/Users/steve/work/code/finbourne-notebooks/.venv/lib/python3.13/site-packages/finbourne_sdk_utils/cocoon/cocoon.py:1721: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,55,0,0


### 1.2 Portfolio

Create a portfolio and populate with simulated holdings for the Instruments loaded above

In [7]:
portfolios = portfolios_api.list_portfolios_for_scope(
    scope=scope,
    filter=f"id.code eq '{portfolio_code}'"
)

if portfolios.values:
    print(f"Portfolio '{portfolio_code}' already exists in scope '{scope}' - deleting")
    portfolios_api.delete_portfolio(
        scope=scope,
        code=portfolio_code)
 
create_portfolio_request = lm.CreateTransactionPortfolioRequest(
    instrument_scopes=[scope],
    display_name="Realtime Test Portfolio",
    description="Realtime Test Portfolio",
    code=portfolio_code,
    created="2020-01-01T00:00:00+00:00",
    base_currency=default_currency,
)

response = transaction_portfolios_api.create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=create_portfolio_request
)


### 1.3 Holdings

In [8]:
# Here we generate a Holdings dataset from the Instrument universe with fake holdings
# For simplicity, always use the same set of random numbers
rand = random.Random()
rand.seed("test")

holdings_df = instruments_df[['swissValorNumber']].copy()
holdings_df['swissValorNumber'] = holdings_df['swissValorNumber'].astype(str)
holdings_df['units'] = [rand.randrange(100,1000) for r in range(0,len(holdings_df))]
holdings_df['cost'] = [rand.randrange(1000,10000) for r in range(0,len(holdings_df))]
holdings_df['price'] = holdings_df['cost'] / holdings_df['units']

#holdings_df

In [ ]:
holdings_mapping = {
    "identifier_mapping": {
        "ClientInternal": "swissValorNumber"
    },
    "required": {
        "effective_at": "$2020-01-01",
        "code": f"${portfolio_code}",
        "tax_lots.units": "units",
        "tax_lots.portfolio_cost": "cost",
        "tax_lots.cost.currency": f"${default_currency}",
        "tax_lots.cost.amount": "cost"
    },
    "optional": {}
}

result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    file_type="holdings",
    identifier_mapping=holdings_mapping["identifier_mapping"],
)

succ, failed = cocoon.format_holdings_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

/Users/steve/work/code/finbourne-notebooks/.venv/lib/python3.13/site-packages/finbourne_sdk_utils/cocoon/cocoon.py:1721: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,1,0,0


### 1.4 Prices

Load prices here to fall back on if the feed is not streaming - typically because the market is closed.

In [ ]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": f"${quotes_instrument_id}",
    "quote_id.effective_at": "$2024-02-01",
    "quote_id.quote_series_id.provider": f"${quotes_supplier}",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "swissValorNumber",
    "quote_id.quote_series_id.field" : f"${quotes_field}",
    "metric_value.unit": f"${default_currency}",
    "metric_value.value": "price"
}

result = cocoon.load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=holdings_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes",
)

succ, failed, errors = cocoon.format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/Users/steve/work/code/finbourne-notebooks/.venv/lib/python3.13/site-packages/finbourne_sdk_utils/cocoon/cocoon.py:1721: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,55,0,0


### 1.5 Exchange Rates

In [11]:
# Create dummy rates from GBP and CHF
rates = [
    ['GBP/USD', 1.26], 
    ['GBP/EUR', 1.17], 
    ['USD/CHF', 0.87],
]
 
instrument_id_type = "CurrencyPair"
value = "Rate"

rates_df = pd.DataFrame(rates, columns=[instrument_id_type, value])
 
rates_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$CurrencyPair",
    "quote_id.effective_at": "$2024-02-01",
    "quote_id.quote_series_id.provider": f"${quotes_supplier}",
    "quote_id.quote_series_id.quote_type": "$Rate",
    "quote_id.quote_series_id.instrument_id": instrument_id_type,
    "quote_id.quote_series_id.field" : f"${quotes_field}",
    "metric_value.unit": instrument_id_type,
    "metric_value.value": value
}


result = cocoon.load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=rates_df,
        mapping_required=rates_mapping,
        mapping_optional={},
        file_type="quotes",
)

succ, failed, errors = cocoon.format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/Users/steve/work/code/finbourne-notebooks/.venv/lib/python3.13/site-packages/finbourne_sdk_utils/cocoon/cocoon.py:1721: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


,success,failed,errors
0,3,0,0


## 2. Run valuation

In [12]:
configuration_recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                # First source is real-time
                lm.MarketDataKeyRule(
                    key="Quote.SixValoren_BC.*",
                    supplier="SIX",
                    quote_type="Price",
                    data_scope="LUSID-SIX",
                    field="Last",
                    price_source="realtime",
                    source_system="SIX/Streaming",
                    quote_interval="1Y.Live"
                ),
                # Fall back to stored quotes
                lm.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier=quotes_supplier,
                    quote_type="Price",
                    data_scope=scope,
                    field=quotes_field,
                    quote_interval="10Y.0D"
                ),
                # Fall back to stored FX rates
                lm.MarketDataKeyRule(
                    key='Fx.CurrencyPair.*',
                    data_scope=scope,
                    supplier='Lusid',
                    quote_type='Rate',
                    quote_interval='10Y.0D',
                    field="mid"
                )
            ],
            options=lm.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="LusidInstrumentId",
                default_scope=scope,
                attempt_to_infer_missing_fx=True
            ),
        ),
        pricing=lm.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

In [13]:
# Pull the data aggregation by passing the effectiveAt date
def aggregation_request(effectiveAt):
    return lm.ValuationRequest( 
        recipe_id = lm.ResourceId(
            scope = scope,
            code = recipe_code
        ),
        metrics = [
            lm.AggregateSpec(key="Instrument/default/Name", op="Value"),
            lm.AggregateSpec(key="Valuation/PV", op="Proportion"),
            lm.AggregateSpec(key="Valuation/PV", op="Sum"),
            lm.AggregateSpec(key="Holding/default/Units", op="Sum"),
            lm.AggregateSpec(key="Quotes/Price", op="Value"),
            lm.AggregateSpec(key="Quotes/Price/EffectiveAt", op="Value"),
        ],
        group_by=["Instrument/default/Name"],
        # choose the valuation date for the request - set using effectiveAt
        valuation_schedule=lm.ValuationSchedule(effective_at=effectiveAt),
        portfolio_entity_ids = [lm.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )]
        )

aggregation = aggregation_api.get_valuation(
  valuation_request=aggregation_request(effectiveAt=datetime.now(pytz.UTC).isoformat())
)
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PV),Sum(Valuation/PV),Sum(Holding/default/Units),Quotes/Price,Quotes/Price/EffectiveAt
0,Allegion/RegSh USD0.01,None,"93,684.21",771.00,121.51,2025-04-07T14:52:49.5799780+00:00
1,Eaton Corp/RegSh USD0,None,"238,101.32",958.00,248.54,2025-04-07T14:52:48.2610290+00:00
2,Trane Tech/RegSh USD1,None,"268,388.52",852.00,315.01,2025-04-07T14:50:28.0638770+00:00
3,Pentair/RegSh USD0.01,None,"76,782.58",982.00,78.19,2025-04-07T14:51:52.4792130+00:00
4,TE Connectiv/RegSh CHF0.57,None,"6,918.00",356.00,19.43,2024-02-01T00:00:00.0000000+00:00
5,Accenture/RegSh Cl-A USD0.0000225,None,"212,115.00",750.00,282.82,2025-04-07T14:52:34.7639420+00:00
6,Johnson Ctr Int/RegSh USD0.01,None,"62,530.10",877.00,71.30,2025-04-07T14:52:49.5514050+00:00
7,Amcor/RegSh USD0.01,None,"3,043.00",340.00,8.95,2025-04-07T14:52:46.1045680+00:00
8,Aptiv/RegSh USD0.01,None,"1,791.00",292.00,6.13,2024-02-01T00:00:00.0000000+00:00
9,STERIS/RegSh USD0.001,None,"29,560.65",141.00,209.65,2025-04-07T14:44:55.9203720+00:00


In [14]:
display(HTML("<h1>Links</h1>"))

display(HTML(f'''
  <a href="{api_url}app/dashboard/holdings?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&recipeCode={recipe_code}"
  target="_blank">
    Holdings
  </a>'''))

display(HTML(f'''
  <a href="{api_url}app/dashboard/valuations?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&recipeCode={recipe_code}"
  target="_blank">
    Valuation with real-time prices
  </a>'''))